In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [49]:
#He implementado try:, except: en la obtención de cada categoría, consejo de Dmitry
def obtener_detalles_inmueble(url_inmueble):
    try:
        response = requests.get(url_inmueble)
        soup = BeautifulSoup(response.text, "html.parser")
        
        try:
            nombre = soup.find("h1", class_="ad-title")
            nombre = nombre.text.strip() if nombre else "Error al acceder al inmueble"
        except:
            nombre = "Error al acceder al inmueble"

        try:
            agencia = soup.find("p", class_="owner-info__name")
            agencia = agencia.find("a").text.strip() if agencia and agencia.find("a") else "Agencia no disponible"
        except:
            agencia = "Agencia no disponible"

        try:
            features_summary = soup.find("ul", class_="features-summary")
            precio_m2 = None
            if features_summary:
                items = features_summary.find_all("li", class_="features-summary__item")
                for item in items:
                    if "€/m²" in item.text:
                        precio_m2 = item.text.strip()
                        break
            precio_m2 = precio_m2 if precio_m2 else "Error al procesar precio m²"
        except:
            precio_m2 = "Error al procesar precio m²"
        
        try:
            precio = soup.find("div", class_="price__value jsPriceValue")
            precio = precio.text.strip() if precio else "Error al procesar precio"
        except:
            precio = "Error al procesar precio"
        
        try:
            superficie = soup.find("span", class_="features__value")
            superficie = superficie.text.strip() if superficie else "Error al procesar superficie"
        except:
            superficie = "Error al procesar superficie"
        
        try:
            actualizacion = soup.find("div", class_="details__block last-update")
            actualizacion = actualizacion.text.strip().replace("Última actualización\n", "").strip() if actualizacion else "Error al procesar actualizacion"
        except:
            actualizacion = "Error al procesar actualizacion"

        try:
            consumo = soup.find_all("span", class_="energy-certificate__tag")
            consumo_etiquetas = []
            for etiqueta in consumo:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_consumo = etiqueta['class'][1].split('--')[1].strip()
                    if letra_consumo in ["a", "b", "c", "d", "e", "f", "g"]: 
                        consumo_etiquetas.append(letra_consumo.upper())
            consumo = ', '.join(consumo_etiquetas) if consumo_etiquetas else "Error al procesar consumo"
        except:
            consumo = "Error al procesar consumo"

        try:
            emisiones = soup.find_all("span", class_="energy-certificate__tag")
            emisiones_etiquetas = []
            for etiqueta in emisiones:
                if 'energy-certificate__tag--' in etiqueta['class'][1]:
                    letra_emisiones = etiqueta['class'][1].split('--')[1].strip()
                    if letra_emisiones in ["a", "b", "c", "d", "e", "f", "g"]:
                        emisiones_etiquetas.append(letra_emisiones.upper())
            emisiones = ', '.join(emisiones_etiquetas) if emisiones_etiquetas else "Error al procesar emisiones"
        except:
            emisiones = "Error al procesar emisiones"

        try:
            caracteristicas = soup.find("div", class_="features-container")
            detalles = {}
            if caracteristicas:
                secciones = caracteristicas.find_all("div", class_="features__content")
                for seccion in secciones:
                    features = seccion.find_all("div", class_="features__feature")
                    for feature in features:
                        label = feature.find("span", class_="features__label").text.strip().replace(":", "")
                        value = feature.find("span", class_="features__value").text.strip() if feature.find("span", class_="features__value") else "N/A"
                        detalles[label] = value
        except:
            detalles = {}
            
        return nombre, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles
    except Exception as e:
        return ("Error al acceder al inmueble",) * 9 + ({},)

def obtener_inmuebles_paginas(base_url, paginas=2):
    todos_los_inmuebles = []
    for i in range(1, paginas + 1):
        print(f"Procesando página {i}")
        url_pagina = f"{base_url}{i}/"
        
        try:
            response = requests.get(url_pagina)
            soup = BeautifulSoup(response.text, "html.parser")
            
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                nombre_detalle, agencia, precio_m2, precio, superficie, actualizacion, consumo, emisiones, detalles = obtener_detalles_inmueble(href)
                
                # He creado una nueva columna a partir del enlace en el que scrapeamos que añade la zona de cada inmueble
                # Donde pone (BCN) debería ir cada codigo que asignemos a la provincia o zona concreta que estemos scrapeando
                zona = base_url.split('/')[-2].split('-')[-1] + ' (BCN)'
                
                inmueble = {
                    "nombre": nombre,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "consumo": consumo,
                    "emisiones": emisiones,
                    "zona": zona
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
            
        except Exception as e:
            print(f"Error al procesar la página {i}: {e}")
        
        time.sleep(2)
    
    df_compras = pd.DataFrame(todos_los_inmuebles)
    return df_compras

# Aquí habría que modificar la url según la zona en la que estemos trabajando
base_url = "https://www.pisos.com/venta/pisos-maresme/"
df_compras = obtener_inmuebles_paginas(base_url, paginas=86) #Aquí modificar la cantidad de páginas según la zona en la que estemos trabajando

#df_compras['actualizacion'] = pd.to_datetime(df_compras['actualizacion'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0], format='%d/%m/%Y')

Procesando página 1
Procesando página 2
Procesando página 3
Procesando página 4
Procesando página 5
Procesando página 6
Procesando página 7
Procesando página 8
Procesando página 9
Procesando página 10
Procesando página 11
Procesando página 12
Procesando página 13
Procesando página 14
Procesando página 15
Procesando página 16
Procesando página 17
Procesando página 18
Procesando página 19
Procesando página 20
Procesando página 21
Procesando página 22
Procesando página 23
Procesando página 24
Procesando página 25
Procesando página 26
Procesando página 27
Procesando página 28
Procesando página 29
Procesando página 30
Procesando página 31
Procesando página 32
Procesando página 33
Procesando página 34
Procesando página 35
Procesando página 36
Procesando página 37
Procesando página 38
Procesando página 39
Procesando página 40
Procesando página 41
Procesando página 42
Procesando página 43
Procesando página 44
Procesando página 45
Procesando página 46
Procesando página 47
Procesando página 48
P

In [50]:
df_compras

,nombre,agencia,precio_m2,precio,superficie,href,actualizacion,consumo,emisiones,zona,...,Agua,Soleado,Carpintería interior,Portero automático,Adaptado a personas con movilidad reducida,Luz,Trastero,Tipo de casa,Interior,No se aceptan mascotas
0,Piso en Carrer de l'Estadi,IMMO-NOVA,1.794 €/m²,183.000 €,102 m²,https://www.pisos.com/comprar/piso-cerdanyola_...,Anuncio actualizado el 12/02/2025,Error al procesar consumo,Error al procesar emisiones,maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Casa en Carrer del Milfulles,PROPIETARIS.CAT,1.645 €/m²,329.000 €,200 m²,https://www.pisos.com/comprar/casa-santa_susan...,Anuncio actualizado el 19/02/2025,"E, E","E, E",maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Piso en Ronda de Barceló, 49",PREMIER ESPAÑA,3.230 €/m²,365.000 €,113 m²,https://www.pisos.com/comprar/piso-mataro_eixa...,Anuncio actualizado el 18/02/2025,"A, A","A, A",maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Casa en Carrer de Sant Ramon,homellar,1.274 €/m²,265.000 €,208 m²,https://www.pisos.com/comprar/casa-tordera_nuc...,Anuncio actualizado el 04/02/2025,"G, G","G, G",maresme (BCN),...,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Piso en El Palau,FINQUES SANTOS,2.000 €/m²,220.000 €,110 m²,https://www.pisos.com/comprar/piso-el_palau-40...,Anuncio actualizado el 17/02/2025,Error al procesar consumo,Error al procesar emisiones,maresme (BCN),...,NaN,N/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,Chalet en Vallgorguina,B.C. GESTIONS,1.218 €/m²,164.500 €,135 m²,https://www.pisos.com/comprar/chalet-vallgorgu...,Anuncio actualizado el 07/02/2025,Error al procesar consumo,Error al procesar emisiones,maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2662,Chalet en calle La Baronia del Montseny,CENTURY 21 URBAN,324 €/m²,130.000 €,401 m²,https://www.pisos.com/comprar/chalet-vallgorgu...,Anuncio actualizado el 18/01/2025,Error al procesar consumo,Error al procesar emisiones,maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,Casa en Otras,PREMIUM HOUSES,750 €/m²,750.000 €,1.000 m²,https://www.pisos.com/comprar/casa-vallgorguin...,Anuncio actualizado el 20/02/2025,Error al procesar consumo,Error al procesar emisiones,maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2664,Chalet en Vallgorguina,FINQUES ALPHA,1.268 €/m²,307.000 €,242 m²,https://www.pisos.com/comprar/chalet-vallgorgu...,Anuncio actualizado el 14/02/2025,"E, E","E, E",maresme (BCN),...,NaN,NaN,NaN,NaN,NaN,NaN,N/A,NaN,NaN,NaN


In [48]:
#Guardamos el df resultante en un .csv de cada zona
df_compras.to_csv('maresme.csv', index=False) #Modificar el nombre con la zona en la que estamos trabajando